In [20]:
import pandas as pd
import numpy as np
import matplotlib as plt
import geopandas as gpd
import folium
import networkx as nx
import osmnx as ox

In [16]:
#Extract subnetwork
highway_shapefile = 'Shapefiles/Snelheid_Wegvakken.shp'
network_temp = gpd.read_file(highway_shapefile)
#network_temp.plot()
#extract network A200
a200_network = network_temp.loc[np.where(network_temp['WEGNR_HMP'] == 'A200')]

df = pd.DataFrame()

df['Road_section_id'] = network_temp['WVK_ID']
df['Road_number'] = network_temp['WEGNR_HMP']
df['geometry'] = network_temp['geometry']
df_a200 = df.loc[np.where(df['Road_number'] == 'A200')]

In [24]:
incident = pd.read_csv('incidents19Q3Q4.csv')
incident.rename(columns = {'vild_primair_wegnummer':'Road_number', 
                           'primaire_locatie_lengtegraad':'longitude', 
                           'primaire_locatie_breedtegraad':'latitude'}, inplace=True)
incident_a200 = incident.loc[np.where(incident['Road_number'] == 'A200')]
incident_a200.head()

,Unnamed: 0,id,type,starttime_new,endtime_new,Road_number,longitude,latitude
604,176137,LCM-LCM19200834-IM_1,accident,2019-11-06 17:33:14,2019-11-06 18:49:06,A200,4.695898,52.384174
2160,179250,LCM-LCM19206007-IM_1,vehicle_obstruction,2019-11-14 17:04:51,2019-11-14 18:20:31,A200,4.704684,52.384193
3667,1637,NLRWS_NLSIT002608644_1,general_obstruction,2019-08-19 17:21:00,2019-08-19 18:01:21,A200,4.668809,52.382328
3792,2064,NLRWS_NLSIT002610155_1,general_obstruction,2019-08-23 12:18:00,2019-08-23 12:39:04,A200,4.694430,52.384090
4480,4534,NLRWS_NLSIT002626921_1,general_obstruction,2019-09-20 12:17:00,2019-09-20 12:18:48,A200,4.695898,52.384174


In [17]:
map = folium.Map(location=[52.399190, 4.893658])
gjson = folium.features.GeoJson(
    a200_network,
).add_to(map)
map

In [18]:
df_a200

,Road_section_id,Road_number,geometry
794,220377035,A200,"LINESTRING (110449.754 488953.438, 110497.000 ..."
795,220377036,A200,"LINESTRING (110389.783 488965.590, 110406.657 ..."
981,212377039,A200,"LINESTRING (106330.008 488535.432, 106372.180 ..."
1417,216377015,A200,"LINESTRING (108103.041 488716.665, 108189.063 ..."
1476,218377006,A200,"LINESTRING (109364.803 488863.244, 109400.000 ..."
1538,600120058,A200,"LINESTRING (108238.636 488732.044, 108147.315 ..."
1567,220377032,A200,"LINESTRING (110389.783 488965.590, 110456.187 ..."
2075,600421300,A200,"LINESTRING (106859.921 488600.464, 106829.824 ..."
2213,216377019,A200,"LINESTRING (108103.041 488716.665, 108127.402 ..."
2348,213377020,A200,"LINESTRING (106816.800 488577.940, 106837.876 ..."


In [14]:
a200_network.columns

Index(['Road_section_id', 'Road_number', 'geometry'], dtype='object')